Установка библиотек

In [1]:
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 645.7/645.7 kB 6.6 MB/s eta 0:00:00


In [2]:
import cv2
import os
from ultralytics import YOLO

In [3]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [4]:
!pip install opencv-python
!pip install ultralytics

In [5]:
%pip install ultralytics
import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.0.217 🚀 Python-3.10.12 torch-2.1.0+cu118 CUDA:0 (Tesla V100-SXM2-16GB, 16151MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 26.9/78.2 GB disk)


Загрузка датасета с roboflow

In [6]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="13nMH8aJ47Vi5S6NszrR")
project = rf.workspace("innopolis-university-aurlu").project("vehicles-cd3uu")
dataset = project.version(5).download("yolov8")


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.3/63.3 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.7/178.7 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.2/72.2 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 9.0 MB/s eta 0:00:00
  Attempting uninstall: pyparsing
    Found existing installation: pyparsing 3.1.1
    Uninstalling pyparsing-3.1.1:
      Successfully uninstalled pyparsing-3.1.1
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.8.1.78
    Uninstalling opencv-python-headless-4.8.1.78:
      Successfully uninstalled opencv-python-headless-4.8.1.78
  Attempting uninstall: idna
    Found existing 

loading Roboflow workspace...
loading Roboflow project...
Dependency ultralytics==8.0.196 is required but found version=8.0.217, to fix: `pip install ultralytics==8.0.196`



Extracting Dataset Version Zip to Vehicles-5 in yolov8:: 100%|██████████| 9804/9804 [00:02<00:00, 3866.85it/s]


Обучение модели

In [9]:
# Train YOLOv8n on COCO8 for 3 epochs
!yolo train model=yolov8m.pt data=/content/Vehicles-5/data.yaml epochs=100 imgsz=640 batch=200

Ultralytics YOLOv8.0.217 🚀 Python-3.10.12 torch-2.1.0+cu118 CUDA:0 (Tesla V100-SXM2-16GB, 16151MiB)
engine/trainer: task=detect, mode=train, model=yolov8m.pt, data=/content/Vehicles-5/data.yaml, epochs=100, patience=50, batch=200, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train3, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, stream_buffer=False, line_width=None, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, boxes=True, format=torchscript,

Настройка директорий с неразмеченными видео

In [ ]:
directory = r"/content/drive/MyDrive/train" #
train_images_path = '/content/drive/MyDrive/train'
img_list = os.listdir(directory)
print(f"В папке имеется {len(img_list)} изображений")

В папке имеется 105 изображений


Разметка неразмеченных видео

In [ ]:
import cv2
import os
from ultralytics import YOLO

model = YOLO('/content/best.pt') #загрузка весов модели
directory = r"/content/drive/MyDrive/Word Hackaton/data/1/KRA-2-7-2023-08-23-evening(1)"
train_images_path = '/content/drive/MyDrive/Word Hackaton/data/1/KRA-2-7-2023-08-23-evening(1)'
img_list = os.listdir(directory)
print(f"В папке имеется {len(img_list)} изображений")

for img_name in img_list:
    img_filepath = os.path.join(directory, img_name)
    print(img_filepath)
    img = cv2.imread(img_filepath)

    if img is None:
        continue

    img_copy = img

    # предобработка изображения как перед обучением
    # gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # gray_three = cv2.merge([gray,gray,gray])
    # img = gray_three

    h, w,_  = img.shape

    # получаем предсказания по картинке
    results = model.predict(source=img, conf=0.50)

    # расшифровываем объект results
    bboxes_ = results[0].boxes.xyxy.tolist()
    bboxes = list(map(lambda x: list(map(lambda y: int(y), x)), bboxes_))
    confs_ = results[0].boxes.conf.tolist()
    confs = list(map(lambda x: int(x*100), confs_))
    classes_ = results[0].boxes.cls.tolist()
    classes = list(map(lambda x: int(x), classes_))
    cls_dict = results[0].names
    class_names = list(map(lambda x: cls_dict[x], classes))

    # приводим дешифрированные данные в удобный вид
    annot_lines = []
    for index, val in enumerate(class_names):
        xmin, ymin, xmax, ymax = int(bboxes[index][0]), int(bboxes[index][1]), int(bboxes[index][2]), int(bboxes[index][3])
        width = xmax - xmin
        height = ymax - ymin
        center_x = xmin + (width/2)
        center_y = ymin + (height/2)
        annotation = f"{classes[index]} {center_x/w} {center_y/h} {width/w} {height/h}"
        annot_lines.append(annotation)

    # копируем картинку в папку базы изображений для импорта
    cv2.imwrite(os.path.join(train_images_path , img_name), img_copy)

    # записываем файл аннотации в папку базы изображений для импорта
    txt_name = img_name.replace(".png",".txt")
    with open(f'/content/drive/MyDrive/Word Hackaton/label_new/{txt_name}', 'w') as f:
        for line in annot_lines:
            f.write(line)
            f.write('\n')


FileNotFoundError: ignored

Детекция на видео

In [ ]:
!nvidia-smi

In [ ]:
import os
HOME = os.getcwd()
print(HOME)

In [ ]:
%cd {HOME}
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1pz68D1Gsx80MoPg-_q-IbEdESEmyVLm-' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1pz68D1Gsx80MoPg-_q-IbEdESEmyVLm-" -O vehicle-counting.mp4 && rm -rf /tmp/cookies.txt

Загрузка видео для разметки

In [ ]:
SOURCE_VIDEO_PATH = f"{HOME}/vehicle-counting.mp4" #путь к видео

In [ ]:
# Pip install method (recommended)

!pip install ultralytics

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

In [ ]:
%cd {HOME}
!git clone https://github.com/ifzhang/ByteTrack.git
%cd {HOME}/ByteTrack

!sed -i 's/onnx==1.8.1/onnx==1.9.0/g' requirements.txt

!pip3 install -q -r requirements.txt
!python3 setup.py -q develop
!pip install -q cython_bbox
!pip install -q onemetric
!pip install -q loguru lap thop

from IPython import display
display.clear_output()


import sys
sys.path.append(f"{HOME}/ByteTrack")


import yolox
print("yolox.__version__:", yolox.__version__)

In [ ]:
!pip install supervision==0.1.0


from IPython import display
display.clear_output()


import supervision
print("supervision.__version__:", supervision.__version__)

In [ ]:
from supervision.draw.color import ColorPalette
from supervision.geometry.dataclasses import Point
from supervision.video.dataclasses import VideoInfo
from supervision.video.source import get_video_frames_generator
from supervision.video.sink import VideoSink
from supervision.notebook.utils import show_frame_in_notebook
from supervision.tools.detections import Detections, BoxAnnotator
from supervision.tools.line_counter import LineCounter, LineCounterAnnotator

In [ ]:
from yolox.tracker.byte_tracker import BYTETracker, STrack
from onemetric.cv.utils.iou import box_iou_batch
from dataclasses import dataclass


@dataclass(frozen=True)
class BYTETrackerArgs:
    track_thresh: float = 0.25
    track_buffer: int = 30
    match_thresh: float = 0.8
    aspect_ratio_thresh: float = 3.0
    min_box_area: float = 1.0
    mot20: bool = False

In [ ]:
from typing import List

import numpy as np

def detections2boxes(detections: Detections) -> np.ndarray:
    return np.hstack((
        detections.xyxy,
        detections.confidence[:, np.newaxis]
    ))

def tracks2boxes(tracks: List[STrack]) -> np.ndarray:
    return np.array([
        track.tlbr
        for track
        in tracks
    ], dtype=float)

def match_detections_with_tracks(
    detections: Detections,
    tracks: List[STrack]
) -> Detections:
    if not np.any(detections.xyxy) or len(tracks) == 0:
        return np.empty((0,))

    tracks_boxes = tracks2boxes(tracks=tracks)
    iou = box_iou_batch(tracks_boxes, detections.xyxy)
    track2detection = np.argmax(iou, axis=1)

    tracker_ids = [None] * len(detections)

    for tracker_index, detection_index in enumerate(track2detection):
        if iou[tracker_index, detection_index] != 0:
            tracker_ids[detection_index] = tracks[tracker_index].track_id

    return tracker_ids

Загрузка предобученных весов

In [ ]:
MODEL = "yolov8x.pt" #путь к весам

In [ ]:
from ultralytics import YOLO

model = YOLO(MODEL)
model.fuse()

In [ ]:
CLASS_NAMES_DICT = {
    2: 'bus',
    5: 'car',
    7: 'truck'
} #данные из data.yaml файлав

CLASS_ID = [2, 5, 7]

In [ ]:
generator = get_video_frames_generator(SOURCE_VIDEO_PATH)

box_annotator = BoxAnnotator(color=ColorPalette(), thickness=4, text_thickness=4, text_scale=2)
iterator = iter(generator)
frame = next(iterator)

results = model(frame)
detections = Detections(
    xyxy=results[0].boxes.xyxy.cpu().numpy(),
    confidence=results[0].boxes.conf.cpu().numpy(),
    class_id=results[0].boxes.cls.cpu().numpy().astype(int)
)

labels = [
    f"{CLASS_NAMES_DICT[class_id]} {confidence:0.2f}"
    for _, confidence, class_id, tracker_id
    in detections
]

frame = box_annotator.annotate(frame=frame, detections=detections, labels=labels)

%matplotlib inline
show_frame_in_notebook(frame, (16, 16))

Проверка меток над машинами

In [ ]:
LINE_START = Point(50, 1500)
LINE_END = Point(3840-50, 1500)

TARGET_VIDEO_PATH = f"{HOME}/vehicle-counting-result.mp4"

In [ ]:
VideoInfo.from_video_path(SOURCE_VIDEO_PATH)

In [ ]:
from tqdm.notebook import tqdm

byte_tracker = BYTETracker(BYTETrackerArgs())
video_info = VideoInfo.from_video_path(SOURCE_VIDEO_PATH)
generator = get_video_frames_generator(SOURCE_VIDEO_PATH)
line_counter = LineCounter(start=LINE_START, end=LINE_END)
box_annotator = BoxAnnotator(color=ColorPalette(), thickness=4, text_thickness=4, text_scale=2)
line_annotator = LineCounterAnnotator(thickness=4, text_thickness=4, text_scale=2)

with VideoSink(TARGET_VIDEO_PATH, video_info) as sink:
    for frame in tqdm(generator, total=video_info.total_frames):
        results = model(frame)
        detections = Detections(
            xyxy=results[0].boxes.xyxy.cpu().numpy(),
            confidence=results[0].boxes.conf.cpu().numpy(),
            class_id=results[0].boxes.cls.cpu().numpy().astype(int)
        )
        mask = np.array([class_id in CLASS_ID for class_id in detections.class_id], dtype=bool)
        detections.filter(mask=mask, inplace=True)
        tracks = byte_tracker.update(
            output_results=detections2boxes(detections=detections),
            img_info=frame.shape,
            img_size=frame.shape
        )
        tracker_id = match_detections_with_tracks(detections=detections, tracks=tracks)
        detections.tracker_id = np.array(tracker_id)
        mask = np.array([tracker_id is not None for tracker_id in detections.tracker_id], dtype=bool)
        detections.filter(mask=mask, inplace=True)
        labels = [
            f"#{tracker_id} {CLASS_NAMES_DICT[class_id]} {confidence:0.2f}"
            for _, confidence, class_id, tracker_id
            in detections
        ]
        line_counter.update(detections=detections)
        frame = box_annotator.annotate(frame=frame, detections=detections, labels=labels)
        line_annotator.annotate(frame=frame, line_counter=line_counter)
        sink.write_frame(frame)

Подсчет времени с помощью ResNet50

In [ ]:
import os
import json
import cv2
import numpy as np
import pandas as pd

def read_video_frames(video_path):
    video_frames = []
    cap = cv2.VideoCapture(video_path)
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        video_frames.append(frame)
    return video_frames

def load_json_file(json_path):
    with open(json_path) as file:
        data = json.load(file)
    return data

def detect_objects(video_frames, json_data):
    objects_count = {'car': 0, 'van': 0, 'bus': 0}
    objects_duration = {'car': 0, 'van': 0, 'bus': 0}
    zone_length = 20

    for frame, annotation in zip(video_frames, json_data):
        for obj in annotation['objects']:
            if obj['class'] in ['car', 'van', 'bus']:
                xmin = obj['bounding_box']['xmin']
                xmax = obj['bounding_box']['xmax']
                ymin = obj['bounding_box']['ymin']
                ymax = obj['bounding_box']['ymax']
                area_length = xmax - xmin
                if zone_length >= area_length:
                    class_type = obj['class']
                    objects_count[class_type] += 1
                    objects_duration[class_type] += 1

    return objects_count, objects_duration

def calculate_average_speed(objects_count, objects_duration, video_duration):
    df_data = []
    zone_length = 20
    for class_type in objects_count:
        count = objects_count[class_type]
        duration = objects_duration[class_type]
        if duration != 0:
            average_speed = zone_length * count / (duration * video_duration)
        else:
            average_speed = 0
        df_data.append({'Class': class_type, 'Count': count, 'Average Speed': average_speed})
    return df_data

#Ускорение работы распараллеливание
def main():
    video_folder = '/path/to/video/folder' #пути к папкам
    json_folder = '/path/to/json/folder' #пути к папкам
    output_csv_file = '/path/to/output.csv' #путь с названием выходного файла

    df_data = []
    for file_name in os.listdir(video_folder):
        video_path = os.path.join(video_folder, file_name)
        json_path = os.path.join(json_folder, file_name.replace('.mp4', '.json'))

        if os.path.isfile(json_path):
            video_frames = read_video_frames(video_path)
            json_data = load_json_file(json_path)
            video_duration = len(video_frames)

            objects_count, objects_duration = detect_objects(video_frames, json_data)
            df_data.extend(calculate_average_speed(objects_count, objects_duration, video_duration))

    df = pd.DataFrame(df_data)
    df.insert(0, 'Video', [os.path.splitext(file)[0] for file in os.listdir(video_folder) if file.endswith('.mp4')])
    df.to_csv(output_csv_file, index=False)

if name == 'main':
    main()
